# ICARUS Signal Processing Chain NoteBook  
  
### 1. Introduction
  * Reading data and event viewing
  * Basic introduction to ICARUS induction wire data and signal processing
  
### 2. Coherent Noise Correction
  * Morphological Filtering
  * ROI Selection
  * Coherent Noise Correction
  
### 3. Adaptive Wiener Denoising
  * Lee filter
  * MMWF and MMWFStar
  * Weighted Lee filter
  
### 4. Deconvolution
  * 1D deconvolution
    * Inverse Filters
  * 2D deconvolution

### 1. Introduction

In [ ]:
%matplotlib inline
import uproot
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS
# Make sure the paths are correct
import os,sys
# Start by defining paths to base installations
baseDir = "/u/nu/koh0207"
icarusSigProcPath = baseDir + "/icarus-sigproc-tools"
sigProcToolsPath = baseDir + "/signal_processing"

# Now update the PYTHONPATH
sys.path.insert(0,icarusSigProcPath)
sys.path.insert(0,sigProcToolsPath+"/python")

# Next update the path to the code
os.environ['LD_LIBRARY_PATH'] = sigProcToolsPath+"/build/lib" + os.pathsep + os.environ['LD_LIBRARY_PATH']

###############################################################################
# Set the path to and name of the data file to be read
PATHNAME       = "/gpfs/slac/staas/fs1/g/neutrino/usher"
RECOFILENAME   = PATHNAME + "/data_run621_1_EW05_PED_20190926T201608_DECODE.root"
#RECOFILENAME   = PATHNAME + "/data_run618_2_EW01M_PED_20190926T174904_DECODE.root"

In [ ]:
# Importing python-cpp interfacing modules for testing C++ implementations.
from sproc import sproc
from sigproc_tools.sigproc_objects.filterevents import FilterEvents
from sigproc_tools.sigproc_utils.denoising import Denoiser
from sigproc_tools.sigproc_utils.morphology import *

In [ ]:
# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events" 
DAQNAME        = "raw::RawDigits_daq__TPCANALYSIS."

# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)
#print(data_file.keys(),"\n------\n")
print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]
#print(events_folder)

# Go ahead and filter the events (subtracting pedestals)
noiseFilter = FilterEvents(events_folder,DAQNAME)
nChannelsPerGroup = 64   # <==== We choose this
numChannels       = noiseFilter.rawdigits.numChannels(0)
numGroups         = numChannels // nChannelsPerGroup
numEvents = noiseFilter.filterEvents(nChannelsPerGroup)
print("Noise processing complete with",numEvents)

Generate `FullResponse` objects for collection, first induction, and middle induction planes.

In [ ]:
ls ../data

In [ ]:
from sigproc_tools.sigproc_objects.fullresponse        import FullResponse
from sigproc_tools.sigproc_objects.fieldresponse       import FieldResponse
from sigproc_tools.sigproc_objects.electronicsresponse import ElectronicsResponse
from sigproc_tools.sigproc_objects.filter              import *

numTimeBins  = 4096
samplingRate = 0.4

# Here the input parameters are defined. 
# Generally, the gaussians are defined by just their offset (which is almost always set to 0) and their width. The values input are in kHz. 
# The Wiener filter will be defined by its offset (again, mostly 0), its scaling factor (sort of sigma for the Gaussian) and exponent. Obviously,
# the Wiener function will revert to a gaussian if the exponent is 2. 
paramsInd0   = [(0.,1.),(0.,90.)]     #[(0.,4.0), (0.,100.)]
paramsInd1   = [(0.,5.),(0.,90.)]   #[(0.,4.0), (0.,100.)]
paramsCol    = (0.,90.)               #(0.,100.)
wienerInd    = (0.,60.,1.7)
gaussWiener  = [(0.,2.),(0.,100.,3.)]

# Here we build the filter functions 
filterInduction0  = FilterDoubleGauss(numTimeBins,samplingRate,paramsInd0)
filterInduction1  = FilterDoubleGauss(numTimeBins,samplingRate,paramsInd1)
filterWiener      = FilterPseudoWiener(numTimeBins,samplingRate,wienerInd)
filterCollection  = FilterGauss(numTimeBins,samplingRate,paramsCol)
filterGaussWiener = FilterGaussPseudoWiener(numTimeBins,samplingRate,gaussWiener)

inputFilePath = "../data/"
TPCresponses = [None, None, None]
TPCresponses[2] = FullResponse(inputFilePath,"t600_response_vw02_v0.0.root",filterCollection)
TPCresponses[1] = FullResponse(inputFilePath,"t600_response_vw01_v0.0.root",filterInduction1,TPCresponses[2].FieldResponse.normFactor)
TPCresponses[0] = FullResponse(inputFilePath,"t600_response_vw00_v0.0.root",filterInduction0,TPCresponses[2].FieldResponse.normFactor)

Configurations for generating toy event overlay:

In [ ]:
from sigproc_tools.sigproc_functions.fakeParticle      import genWhiteNoiseWaveform,genSpikeWaveform,createParticleTrajectory
from sigproc_tools.sigproc_objects.filter              import *
from scipy.optimize import curve_fit
import scipy.signal as sig
import math

# Experiment with creating an event pic
# Start with "just" a charge deposit on a zero waveform
numElectrons = 10000 # number electrons per mm after recombination
numChannels  = 576
numTicks     = 4096

angleToWire  = 45 # degrees
slope        = math.tan(math.radians(90-angleToWire)) / 0.213
startTick    = 1000

startWire    = 50
stopWire     = 550

responseType = 1   # 0 for first induction, 1 for middle induction, 2 for collection.
eventNum     = 0
wireNum      = 255

print("Angle to wire:",angleToWire,"(deg), tan(theta):",math.tan(math.radians(90-angleToWire)),", slope:",slope)

wireRange = (startWire,stopWire)
tickRange = (startTick,int(round(slope*(wireRange[1]-wireRange[0])+startTick)))

print("Using wireRange:",wireRange,", tickRange:",tickRange)

We will first examine the chain for the colleciton plane.

In [ ]:
spikeResponse,spikeInput = createParticleTrajectory(TPCresponses[responseType],numElectrons,wireRange,tickRange,(numChannels,numTicks))
fullEvent = spikeResponse + noiseFilter.waveLessPedAll[eventNum,:,:]
print("Overlay event created, wires from:",wireRange,", ticks from:",tickRange)

Plot `fullEvent` to visualize generated event overlayed on top of noise.

In [ ]:
from plotting.graphs import *
makePlots = False
if makePlots:
    overlayPic = plotEventView3D(fullEvent)
    print("Calling the show method")
    overlayPic.show()

### 2. Coherent Noise Correction

For coherent noise correction, we will use the `Denoiser` modules imported from `sigproc_tools.sigproc_utils.denoising`.  
This is a simple class for accessing C++ backend denoising modules, so that one does not have to convert from `np.ndarray` to `ROOT.std.vector` and vice versa.

In [ ]:
denoiser = Denoiser(fullEvent)

The `denoiser` class has a member function `removeCoherentNoise2D` that will perform morphological filtering to select ROIs. 

Once the ROIs are determined, it then removes the coherent noise components for each tick and each channel group (64) from non-signal regions.

The arguments to the `removeCoherentNoise2D` are defined as follows:
  * Morphological Filter Mode: `'d'` (dilation), `'e'` (erosion), `'g'` (gradient). These are the main filters that will be useful for our purposes.
  * structuringElement (tuple): The moving window corresponding to the region used to compute morphological operations. 
  * window (int): when set to a positive number, the denoiser will expand `window / 2` pixels vertically and horizontally for each signal detected via morphological operations. 
  * threshold (float): thresholding value for selecting signals based on morphological filtering. The optimal value may be different for different planes/filters. 

In [ ]:
output = denoiser.removeCoherentNoise2D('g', structuringElement=(16,18), window=10, threshold=8.0)

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(np.array(output))
    print("Calling the show method")
    overlayPic.show()

After running coherent noise removal, the following waveforms are stored as instance attributes:
  * `denoiser.waveLessCoherent`: coherent noise corrected waveforms. 
  * `denoiser.selectVals`: detected signal region from morphological operations.
  * `denoiser.roi`: region of interest. Note that it can be different from `selectVals` if `window != 0`. 
  * `denoiser.morphedWaveforms`: morphological filter applied waveform. 
  * `denoiser.instrinsicRMS`: the intrinsic RMS value for each channel group after coherent noise correction.
  
  
We may plot `denoiser.waveLessCoherent` for visual inspection:

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(np.array(output))
    print("Calling the show method")
    overlayPic.show()

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(denoiser.waveLessCoherent)
    print("Calling the show method")
    overlayPic.show()

### 3. Adaptive Wiener Filtering

To further reduce noise, we can apply adaptive wiener filtering to the coherent noise corrected waveform shown above. Here we survey some of the available filtering techniques. 

The adaptive wiener filtering can be accessed via the `Deconvolver` module defined in `sigproc_tools.sigproc_utils.deconvolution`. However this is likely to be renamed, since the filters described here are not deconvolution algorithm in the sense of deconvolving the response function to reconstruct the original signal. It is better to understand these algorithm as generic noise removing filters.

We construct a `Deconvoler` instance from the coherent noise corrected waveforms and the 2D boolean array containing signal detection information. Yet, not all algorithms available in `Deconvolver` will use the roi information. 

In [ ]:
from sigproc_tools.sigproc_utils.adaptive_filter import AdaptiveFilter
#from skimage.restoration import denoise_tv_bregman
adFilter = AdaptiveFilter(denoiser.waveLessCoherent, selectVals=denoiser.selectVals.astype(bool))

Here we list currently implemented filters, and a rough description of their functionalities:  
  * `lee`: Approximate wiener filter that uses local statistics to reduce additive noise.
  * `MMWF`: same as `lee`, yet this uses median in place of the mean. 
  * `MMWFStar`: same as `MMWF`, but using squared error from the median instead of variance. 
  * `lee_enhanced`: same as `lee`, but with using adaptive pixel-wise weighting.  

Note that despite their name as "adaptive wiener filters" all computations are performed in the spatial domain via a convolution-like procedure (moving window). 

For detailed description of each filters, please consult the LaTeX documentation for signal processing. 

In [ ]:
adFiltWaveform = denoiser.waveLessCoherent
# adFiltWaveform = adFilter.denoise_2d('lee', structuringElement=(7,20))
# adFiltWaveform = adFilter.denoise_2d('MMWF', structuringElement=(7,20))
# adFiltWaveform = adFilter.denoise_2d('MMWFStar', structuringElement=(7,20))
# Collection
# adFiltWaveform = adFilter.denoise_2d('lee_enhanced', structuringElement=(3,3))
# First Induction
# adFiltWaveform = adFilter.denoise_2d('lee_enhanced', structuringElement=(3,3))
# adFiltWaveform = adFiltWaveform.astype(int)
# deconvWaveform = denoise_tv_bregman(denoiser.waveLessCoherent, 0.1)
# deconvWaveform = deconvolver.deconvolve_2d('lee_enhanced_roi', structuringElement=(3,3))

In [ ]:
makePlots = False
if makePlots:
    overlayPic = plotEventView3D(adFiltWaveform.astype(int))
    print("Calling the show method")
    overlayPic.show()

### 4. Deconvolution

After noise removal in the spatial domain, we now reconstuct the original signal by deconvolving the effects of the response function from the observed signal. 

We first plot the full response functions in the frequency domain:

In [ ]:
# Now plot the response functions
responsePlot = go.Figure()

responsePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Response Induction 0",
        x=TPCresponses[0].Filter.frequencyBins,
        y=np.absolute(TPCresponses[0].ResponseFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Response Induction 1",
        x=TPCresponses[1].Filter.frequencyBins,
        y=np.absolute(TPCresponses[1].ResponseFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Response Collection",
        x=TPCresponses[2].Filter.frequencyBins,
        y=np.absolute(TPCresponses[2].ResponseFFT)
    )
])

responsePlot.update_layout(title='$R(t)$', xaxis=dict(range=(0, 600), title='$\omega$'), font=dict(size=15), width=1000)
responsePlot.show()

Since we have access to the response functions, the problem is that of **non-blind deconvolution**; that is, deconvolution with known response function $R(\omega)$. 

The simplest non-blind deconvolution method is **inverse-filtering**, which we examine for the collection plane signal. 

In [ ]:
# Set up for fitting
from sigproc_tools.sigproc_functions.fakeParticle import genWhiteNoiseWaveform,genSpikeWaveform,createParticleTrajectory
from scipy.optimize import curve_fit
import scipy.signal as sig

# Define model function to be used to fit to the data above: 
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

halfRange       = 16
electronicsGain = 67.4
print("spikeResponse shape:",spikeResponse.shape,", spikeInput shape:",spikeInput.shape)

rawWaveform   = spikeResponse[wireNum,:]
spikeWaveform = spikeInput[wireNum,:]
startTick     = np.argmax(spikeWaveform)

if not np.isscalar(startTick):
    startTick = np.rint(np.mean(startTick)).astype(int)
    
chargeInd     = np.where(spikeWaveform>0.9)
totCharge     = np.sum(spikeWaveform[chargeInd]) * ㄹelectronicsGain
print("chargeInd:",chargeInd)
print("spike:",spikeWaveform[chargeInd])
print("Total charge:",totCharge)

The goal is to estimate the total charge of the reconstructed signal as close as possible to the true total charge (given above). 

Running deconvolution on the true waveform without noise gives the following:

In [ ]:
# run  the deconvolution on this waveform
rawWaveformFFT = np.fft.rfft(rawWaveform)
outputWaveformFFT = np.multiply(rawWaveformFFT,TPCresponses[responseType].DeconvolutionFFT)
outputWaveform = np.fft.irfft(outputWaveformFFT)
outputWaveform = np.roll(outputWaveform,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

In [ ]:
tickVals = np.arange(numTicks)
# Now let's try fitting the resulting peak to check our charge resolution
peakValue = outputWaveform[startTick]
peakRange = np.where(outputWaveform > 0.5*peakValue)
print("peakRange:",peakRange,", first/last:",peakRange[0][0],",",peakRange[0][-1])
peakSigma = (peakRange[0][-1]-peakRange[0][0])/2.2
print("peakSigma:",peakSigma)
fitParams = np.array([outputWaveform[startTick],startTick,peakSigma]).astype(np.float64)
fitRange  = (startTick-int(round(3.5*peakSigma)),startTick+int(round(3.5*peakSigma)))
fitXVals  = tickVals[fitRange[0]:fitRange[1]]
fitYVals  = outputWaveform[fitRange[0]:fitRange[1]]
print("fitParams:",fitParams)

# here is where the fit is done
coeff,varMatrix = curve_fit(
    gauss,fitXVals.astype(np.float64),fitYVals.astype(np.float64),p0=fitParams)
print("Fit coefficients - Pulse Height: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])

totalCharge = electronicsGain * coeff[0] * coeff[2] * math.sqrt(2.* math.pi)
print("Charge input:",totCharge,", Charge out:",totalCharge)

fitCurve = gauss(fitXVals,*np.array(coeff))
waveformPlot = go.Figure()
waveformPlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="spikeWaveform",
        x=tickVals,
        y=spikeWaveform,
#         y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Raw Waveform",
        x=tickVals,
        y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=outputWaveform
    ),
    go.Scatter(
        line=dict(color='red',dash='dash',width=1),
        name="Fit",
        x=fitXVals,
        y=fitCurve
    )
])

waveformPlot.show()

We now consider the same procedure with the noise removed waveform from the previous section. 

In [ ]:
# Do again with noise
rawWaveWithNoise = adFiltWaveform[wireNum,:]
noise_var = adFilter.noise_var

In [ ]:
#dWaveWithNoise = sig.gauss_spline(medWaveWithNoiseIn,2)
rawWaveWithNoiseFFT = np.fft.rfft(rawWaveWithNoise)
responseFFT = TPCresponses[responseType].ResponseFFT
# rawWaveOutput = richardson_lucy(rawWaveWithNoise.reshape(-1, 1), TPCresponses[responseType].Response.reshape(-1, 1), iterations=50, clip=True)
wienerFFT = np.conj(responseFFT) / (np.abs(responseFFT)**2 + noise_var**2 / np.abs(rawWaveWithNoiseFFT)**2)
rawWaveOutputFFT = np.multiply(rawWaveWithNoiseFFT,TPCresponses[responseType].DeconvolutionFFT)
rawWaveOutputFFT = np.multiply(rawWaveWithNoiseFFT,wienerFFT)
rawWaveOutput = np.fft.irfft(rawWaveOutputFFT)
rawWaveOutput = np.roll(rawWaveOutput,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))
fitYVals  = rawWaveOutput[fitRange[0]:fitRange[1]]
print(fitYVals)
fitXVals  = tickVals[fitRange[0]:fitRange[1]]
# here is where the fit is done
coeff,varMatrix = curve_fit(
    gauss,fitXVals.astype(np.float64),fitYVals.astype(np.float64),p0=coeff)
print("Fit coefficients - Pulse Height: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])
totalCharge = electronicsGain * coeff[0] * coeff[2] * math.sqrt(2.* math.pi)
print("Charge input:",totCharge,", Charge out:",totalCharge)
fitCurve = gauss(fitXVals,*np.array(coeff))

In [ ]:
# Plot time again
noiseWavePlot = go.Figure()
tickVals = np.arange(numTicks)
noiseWavePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="spikeWaveform",
        x=tickVals,
        y=spikeWaveform,
#         y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="After CNC + Adaptive Wiener",
        x=tickVals,
        y=rawWaveWithNoise
    ),
#     go.Scatter(
#         line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
#         name="Med Filt Waveform",
#         x=tickVals,
#         y=medWaveWithNoise
#     ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=rawWaveOutput
    ),
    go.Scatter(
        line=dict(color='red',dash='dash',width=1),
        name="Fit",
        x=fitXVals,
        y=fitCurve
    )
])
noiseWavePlot.show()
rawResponsePlot = go.Figure()
rawResponsePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Raw Response No Noise",
        x=TPCresponses[0].Filter.frequencyBins,
        y=np.absolute(rawWaveformFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Raw Response w/ Noise",
        x=TPCresponses[1].Filter.frequencyBins,
        y=np.absolute(rawWaveWithNoiseFFT)
    )
])
rawResponsePlot.update_layout(xaxis=dict(range=(0, None), title='$\omega$'), font=dict(size=15))
rawResponsePlot.show()

In [ ]:
def pad_along_axis(array: np.ndarray, target_length, axis=0):

    pad_size = target_length - array.shape[axis]
    axis_nb = len(array.shape)

    if pad_size < 0:
        return array

    npad = [(0, 0) for x in range(axis_nb)]
    npad[axis] = (0, pad_size)

    b = np.pad(array, pad_width=npad, mode='constant', constant_values=0)

    return b

In [ ]:
makePlots = False
if makePlots:
    overlayPic = plotEventView3D(reco_full.astype(int))
    print("Calling the show method")
    overlayPic.show()

### 2D Deconvolution

In [ ]:
response = uproot.open('../garfield-icarus-fnal-commissioning.root')
histY = response['FieldRes_Y']
histU = response['FieldRes_U']
histV = response['FieldRes_V']
responseV, axisV = histV.numpy()
responseY, axisY = histY.numpy()
responseU, axisU = histU.numpy()

def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

responseV_rebinned = rebin(responseV, (21, 1000))
responseU_rebinned = rebin(responseU, (21, 1000))
responseY_rebinned = rebin(responseY, (21, 1000))

In [ ]:
responseU_rebinned.shape

In [ ]:
responseV_np = np.pad(responseV_rebinned, pad_width=[(277, 278), (1548, 1548)]) / np.sum(responseV_np)

In [ ]:
np.sum(responseV_np)

In [ ]:
signalFFT = np.fft.rfft2(adFiltWaveform)
ResponseFFT = np.fft.rfft2(responseV_np)

In [ ]:
wienerFFT = np.conj(ResponseFFT) / (np.abs(ResponseFFT)**2 + 3**2)

In [ ]:
wiener2D = np.fft.irfft2(wienerFFT * signalFFT)

In [ ]:
from sigproc_tools.sigproc_utils.deconvolution import Deconvolution

In [ ]:
# noise_var = estimate_sigma(denoiser.waveLessCoherent, multichannel=False)
deconvolver = Deconvolution(adFiltWaveform, response_fn=TPCresponses[responseType].Response, noise_var=3.0)
wiener1D = deconvolver.deconvolve1D()
wiener1D = denoiser.subtractPedestals(wiener1D.astype(np.float32))
wiener1D = np.array(wiener1D)
# fourierShrinkage1D = deconvolver.fourier_shrinkage_1d(0.01)
# fourierShrinkage1D = denoiser.subtractPedestals(fourierShrinkage1D.astype(np.float32))

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(wiener1D.astype(int))
    print("Calling the show method")
    overlayPic.show()

In [ ]:
# fourierShrinkage1D = deconvolver.fourier_shrinkage_1d(0.005)
# # fourierShrinkage1D = denoiser.subtractPedestals(fourierShrinkage1D.astype(np.float32))
# fourierShrinkage1D = np.asarray(fourierShrinkage1D).astype(int)

In [ ]:
from skimage.restoration import (denoise_wavelet, estimate_sigma)
# noise_var = estimate_sigma(wiener1D, multichannel=False)
# print(noise_var)
adFilter2 = AdaptiveFilter(wiener1D, selectVals=denoiser.selectVals.astype(bool))
# adFilter2.noise_var = noise_var
# deconvWaveform = deconvolver.deconvolve_2d('lee', structuringElement=(7,20))
# deconvWaveform = adFilter2.denoise_2d('MMWF', structuringElement=(7,20))
# adFiltWaveform = adFilter.denoise_2d('MMWFStar', structuringElement=(7,20))
# deconvWaveform = adFilter2.denoise_2d('MMWFStar', structuringElement=(7,20))
# deconvWaveform = AdaptiveFilter(deconvWaveform, selectVals=denoiser.selectVals.astype(bool))
# deconvWaveform = denoise_wavelet(wiener1D, method='VisuShrink')
deconvWaveform = adFilter2.denoise_2d('lee_enhanced', structuringElement=(3,3), a=3, epsilon=0.01)
# adFilter2.noise_var = noise_var
# deconvWaveform = deconvolver.deconvolve_2d('lee', structuringElement=(7,20))
# deconvWaveform = adFilter2.denoise_2d('MMWF', structuringElement=(7,20))
# adFiltWaveform = adFilter.denoise_2d('MMWFStar', structuringElement=(7,20))
# deconvWaveform = adFilter2.denoise_2d('MMWFStar', structuringElement=(7,20))
# deconvWaveform = AdaptiveFilter(deconvWaveform, selectVals=denoiser.selectVals.astype(bool))
# deconvWaveform = denoise_wavelet(wiener1D, method='VisuShrink')

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(deconvWaveform.astype(int))
    print("Calling the show method")
    overlayPic.show()

In [ ]:
noiseWavePlot = go.Figure()
tickVals = np.arange(numTicks)
print(np.arange(0, numChannels)[np.max(deconvWaveform.astype(int), axis=1) > 10])
wireNum = 255

rawWaveOutput = wiener1D[wireNum]
rawWaveOutput = np.roll(rawWaveOutput,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

final = np.roll(deconvWaveform[wireNum],int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

fitYVals  = final[fitRange[0]:fitRange[1]]
print(fitYVals)
fitXVals  = tickVals[fitRange[0]:fitRange[1]]
# here is where the fit is done
coeff,varMatrix = curve_fit(
    gauss,fitXVals.astype(np.float64),fitYVals.astype(np.float64),p0=coeff)
print("Fit coefficients - Pulse Height: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])
totalCharge = electronicsGain * coeff[0] * np.abs(coeff[2]) * math.sqrt(2.* math.pi)
print("Charge input:",totCharge,", Charge out:",totalCharge)
fitCurve = gauss(fitXVals,*np.array(coeff))

noiseWavePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="spikeWaveform",
        x=tickVals,
        y=spikeWaveform,
#         y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="After CNC + Adaptive Wiener",
        x=tickVals,
        y=adFilter.waveLessCoherent[wireNum]
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Final Waveform",
        x=tickVals,
        y=final
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=rawWaveOutput
    ),
#     go.Scatter(
#         line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1),
#         name="Deconvolved Waveform",
#         x=tickVals,
#         y=wiener1D[wireNum]
#     )
    go.Scatter(
        line=dict(color='red',dash='dash',width=1),
        name="Fit",
        x=fitXVals,
        y=fitCurve
    )
])

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(spikeInput)
    print("Calling the show method")
    overlayPic.show()

In [ ]:
from sigproc_tools.sigproc_utils.wavelets import WaveletTransform
from skimage.restoration import (denoise_wavelet, estimate_sigma)

In [ ]:
wt = WaveletTransform(fourierShrinkage1D)

In [ ]:
wt.wt2(levelChannels=4, levelTicks=4)
wt.estimateNoise()
# wt.wwf()
# wt.iwt2(levelChannels=4, levelTicks=6)

In [ ]:
print(wt.noiseEstimates[0][0])
sigma_est = estimate_sigma(fourierShrinkage1D, multichannel=True, average_sigmas=True)
print(sigma_est)

In [ ]:
im_visushrink = denoise_wavelet(fourierShrinkage1D, multichannel=False, convert2ycbcr=False,
                                method='VisuShrink', mode='soft',
                                sigma=sigma_est, rescale_sigma=True)

In [ ]:
th = 7.63303315763574
for i, wc in enumerate(wt.waveletCoeffs):
    print(np.array(wc).shape)
    wc = np.where(np.array(wc) < th, 0, wc)
    wc = sproc.pyutil.as_float32_vector_2d(wc.astype(np.float32))
wt.iwt2(levelChannels=4, levelTicks=4)

In [ ]:
makePlots = True
if makePlots:
    overlayPic = plotEventView3D(im_visushrink.astype(int))
    print("Calling the show method")
    overlayPic.show()

In [ ]:
noiseWavePlot = go.Figure()
tickVals = np.arange(numTicks)

rawWaveOutput = fourierShrinkage1D[wireNum]
rawWaveOutput = np.roll(rawWaveOutput,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

noiseWavePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="spikeWaveform",
        x=tickVals,
        y=spikeWaveform,
#         y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="After CNC + Adaptive Wiener",
        x=tickVals,
        y=adFilter.waveLessCoherent[wireNum]
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Med Filt Waveform",
        x=tickVals,
        y=im_visushrink.astype(int)[wireNum]
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=rawWaveOutput
    ),
#     go.Scatter(
#         line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1),
#         name="Deconvolved Waveform",
#         x=tickVals,
#         y=wiener1D[wireNum]
#     )
#     go.Scatter(
#         line=dict(color='red',dash='dash',width=1),
#         name="Fit",
#         x=fitXVals,
#         y=fitCurve
#     )
])